## Máquina de pintado - Modelos de Simulación
#### - Rivera Juan Matías

Una máquina de pintado de piezas puede procesar una pieza en intervalos aleatorios de 10'' a 20'', las piezas llegan de manera continua cada 17 segundos, si una pieza no es pntada antes de los 30'' es considerada descarte y ya no puede ser utilizada.

¿Cuál es la taza de descarte de este modelo de producción?

Adicional:

-Si llega una pieza y ya hay 10 piezas en cola, ésta última es automáticamente descartada.

-Si se rompe la máquina cada hora y se demora un minuto en repararse.

In [52]:
import datetime
import random

##### Función transforma un datetime en un string con valor igual al tiempo(time)

In [53]:
def transformarTiempo(tiempo):
    st = datetime.datetime.strptime(str(tiempo), "%Y-%m-%d %H:%M:%S")
    return st.time()

#### Clase de cola

In [54]:
class Cola:
    def __init__(self,q,TiempoLlegada,DeltaLlegada,TiempoAbandono,DeltaAbandono,ListaAbandono,Limite):
        self.q=q
        self.descarte = 0
        self.tiempoL=TiempoLlegada
        self.deltaL = DeltaLlegada
        self.deltaA = DeltaAbandono
        self.contador = 0
        self.listaAbandono = ListaAbandono
        self.proxAbandono = TiempoAbandono
        self.limite = Limite
        
    def Llegada(self,PS):
        tiempoA = self.tiempoL
        if PS.estado == 0:
            PS.estado = 1
            PS.tiempoFS = self.tiempoL + PS.deltaFS[PS.contador]
            PS.contador+=1
        else:
            if self.q<=10:
                self.q += 1
                self.listaAbandono.append(tiempoA + self.deltaA)
                self.proxAbandono = self.listaAbandono[0]
            else:
                self.descarte += 1

        self.tiempoL += self.deltaL[self.contador]
        self.contador +=1
        return tiempoA
    
    def ReinicioContador(self):
        if self.contador > len(self.deltaL)-1:
            self.contador = 0
            
    def AbandonoDeCola(self):
        tiempoA = self.proxAbandono
        self.q-=1
        del self.listaAbandono[0]
        self.descarte += 1
        if self.listaAbandono:
            self.proxAbandono = self.listaAbandono[0]
        else:
            self.proxAbandono = self.limite
        return tiempoA

#### Clase de Puesto de Servicio

In [55]:
class PuestoDeServicio:
    def __init__(self,Disposicion,Estado,TiempoFS,DeltaFS,TiempoInh,TiempoHab,DeltaDesc,DeltaInh,Limite):
        self.disposicion = Disposicion
        self.estado = Estado
        self.tiempoFS = TiempoFS
        self.deltaFS = DeltaFS
        self.contador = 0
        self.limite = Limite
        self.tiempoInh = TiempoInh
        self.tiempoHab = TiempoHab
        self.deltaDesc = DeltaDesc
        self.deltaInh = DeltaInh
        
    def FinDeServicio(self,Cola):
        tiempoA = self.tiempoFS
        
        if Cola.q > 0:
            Cola.q -= 1
            self.tiempoFS += self.deltaFS[self.contador]
            self.contador += 1
            del Cola.listaAbandono[0]
        else:
            self.estado = 0
            self.tiempoFS = self.limite
        if Cola.listaAbandono:
            Cola.proxAbandono = Cola.listaAbandono[0]
        else:
            Cola.proxAbandono = self.limite
            
        return tiempoA
        
    def ReinicioContador(self):
        if self.contador > len(self.deltaFS)-1:
            self.contador = 0
            random.shuffle(self.deltaFS)
            
    def Inhabilitar(self):
        tiempoA = self.tiempoInh
        self.tiempoFS += deltaDesc
        self.tiempoHab = tiempoA + deltaDesc
        self.disposicion = 0
        self.tiempoInh = self.limite
        return tiempoA       
        
    def Habilitar(self):
        tiempoA = self.tiempoHab
        self.tiempoInh = tiempoA + self.deltaInh
        self.tiempoHab = self.limite
        self.disposicion = 1
        return tiempoA


##### Función para imprimir Tiempo, recibe variables en el siguiente orden: Hora Actual, LLegada del Cliente, Fin de Servicio, N° Cola (q) y Estado del Puesto de Servicio(PS)'

In [62]:
def imprimirTiempo(Cola,PS,TiempoA):
    
    tiempoActual = transformarTiempo(TiempoA)
    tiempoLlegada = transformarTiempo(Cola.tiempoL)
    tiempoFinServicio = transformarTiempo(PS.tiempoFS)
    tiempoInhabilitar = transformarTiempo(PS.tiempoInh)
    tiempoHabilitar = transformarTiempo(PS.tiempoHab)
    tiempoAbandono = transformarTiempo(Cola.proxAbandono)
    if PS.estado==0:
        tiempoFinServicio = "        "
    if Cola.q == 0:
        tiempoAbandono = "        "
    if PS.disposicion==0:
        tiempoInhabilitar = "        "
    else:
        tiempoHabilitar = "        "

    print(tiempoActual," | ",tiempoLlegada," | ",tiempoFinServicio," | ",tiempoAbandono," | ",tiempoInhabilitar," | ",tiempoHabilitar," | ",Cola.q," | ",PS.estado," | ",PS.disposicion," | ",Cola.descarte)

##### Función vector de deltas de tiempo por segundo'

In [63]:
def delta(tiemposDelta):
    listaDelta = []
    for i in tiemposDelta:
        listaDelta.append(datetime.timedelta(seconds = i))
    return listaDelta

##### Función asignación de tiempo'

In [64]:
def tiempo(horas,minutos,segundos):
    tiempo = datetime.datetime.combine(datetime.datetime.today(),datetime.time(horas,minutos,segundos,0)) 
    return tiempo

In [65]:
def timedelta(i):
    return datetime.timedelta(seconds = i)

##### Declaración de los tiempos iniciales y los deltas

In [66]:
#Instancia del limite(HORA DE SERVICIO)'
limite = tiempo(12,0,0)

#Instancia de los tiempos iniciales
tiempoActual = tiempo(8,0,0)
tiempoL = tiempo(8,0,17)
tiempoFS = tiempo(8,0,10)
tiempoInh = tiempoActual + timedelta(360)
tiempoHab = limite




#Tiempos delta particulares del práctico
tiempoDL = [17]
tiempoDFS = [10,11,12,13,14,15,16,17,18,19,20]
listaAb = [tiempo(8,3,30),tiempo(8,4,0),tiempo(8,4,30)]
tiempoAb = listaAb[0]

deltaL = delta(tiempoDL)
deltaFS = delta(tiempoDFS)
deltaAb = timedelta(30)
deltaDesc = timedelta(60)
deltaInh = timedelta(60*60*1)


random.shuffle(deltaFS)

#Instancia de la Cola y del Puesto de Servicio
cola = Cola(3,tiempoL,deltaL,tiempoAb,deltaAb,listaAb,limite+timedelta(30))
PS = PuestoDeServicio(1,1,tiempoFS,deltaFS,tiempoInh,tiempoHab,deltaDesc,deltaInh,limite+timedelta(30))


##### Programa principal, un while que imprime toda la matriz.

In [67]:
print("  Actual  |  LLegada   | Fin de S.  | Abandono   |  Descanso  |  Vuelta T. |  q  |  PS |  S | Descarte")
print("------------------------------------------------------------------------------------------------------")

#Se imprime primero el estado inicial
imprimirTiempo(cola,PS,tiempoActual)


while tiempoActual < limite:
        #Si el tiempo de llegada de cliente es menor al fin de servicio'
        if cola.tiempoL <= PS.tiempoFS  and cola.tiempoL <= cola.proxAbandono and cola.tiempoL < limite and cola.tiempoL <= PS.tiempoInh:
            
            tiempoActual = cola.Llegada(PS)  
            imprimirTiempo(cola,PS,tiempoActual)
            
        #Si el tiempo de fin de servicio es menor que el de llegada'
        elif PS.tiempoFS <= cola.tiempoL and PS.tiempoFS <= cola.proxAbandono and PS.tiempoFS < PS.tiempoInh and PS.tiempoFS < limite and PS.estado == 1:
            tiempoActual = PS.FinDeServicio(cola)
        
            imprimirTiempo(cola,PS,tiempoActual)
         
        elif cola.proxAbandono <= cola.tiempoL and cola.proxAbandono <= PS.tiempoFS and cola.proxAbandono <= PS.tiempoInh and cola.proxAbandono < limite:
            tiempoActual = cola.AbandonoDeCola()
            imprimirTiempo(cola,PS,tiempoActual)
        
        elif PS.tiempoInh <= cola.proxAbandono and PS.tiempoInh <= PS.tiempoFS and PS.tiempoInh <= cola.tiempoL:
            tiempoActual = PS.Inhabilitar()
            imprimirTiempo(cola,PS,tiempoActual)
            
            while cola.tiempoL <= PS.tiempoHab:
                tiempoActual = cola.Llegada(PS)
                imprimirTiempo(cola,PS,tiempoActual)
                cola.ReinicioContador()
                PS.ReinicioContador()
            tiempoActual = PS.Habilitar()
            imprimirTiempo(cola,PS,tiempoActual)
            
        #Cierre
        else:
            if PS.estado == 1:
                print("El último cliente en el PS saldrá a las horas:",transformarTiempo(PS.tiempoFS))
            break
        
        #Reinicio de contadores
        cola.ReinicioContador()
        PS.ReinicioContador()

print("Clientes sin atención:",cola.q)
print("Descartado: %d" % (cola.descarte))
    

  Actual  |  LLegada   | Fin de S.  | Abandono   |  Descanso  |  Vuelta T. |  q  |  PS |  S | Descarte
------------------------------------------------------------------------------------------------------
08:00:00  |  08:00:17  |  08:00:10  |  08:03:30  |  08:06:00  |            |  3  |  1  |  1  |  0
08:00:10  |  08:00:17  |  08:00:23  |  08:04:00  |  08:06:00  |            |  2  |  1  |  1  |  0
08:00:17  |  08:00:34  |  08:00:23  |  08:04:00  |  08:06:00  |            |  3  |  1  |  1  |  0
08:00:23  |  08:00:34  |  08:00:38  |  08:04:30  |  08:06:00  |            |  2  |  1  |  1  |  0
08:00:34  |  08:00:51  |  08:00:38  |  08:04:30  |  08:06:00  |            |  3  |  1  |  1  |  0
08:00:38  |  08:00:51  |  08:00:48  |  08:00:47  |  08:06:00  |            |  2  |  1  |  1  |  0
08:00:47  |  08:00:51  |  08:00:48  |  08:01:04  |  08:06:00  |            |  1  |  1  |  1  |  1
08:00:48  |  08:00:51  |  08:00:59  |            |  08:06:00  |            |  0  |  1  |  1  |  1
08:00:51  

08:15:06  |  08:15:18  |  08:15:20  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:15:18  |  08:15:35  |  08:15:20  |  08:15:48  |  09:07:00  |            |  1  |  1  |  1  |  3
08:15:20  |  08:15:35  |  08:15:30  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:15:30  |  08:15:35  |            |            |  09:07:00  |            |  0  |  0  |  1  |  3
08:15:35  |  08:15:52  |  08:15:48  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:15:48  |  08:15:52  |            |            |  09:07:00  |            |  0  |  0  |  1  |  3
08:15:52  |  08:16:09  |  08:16:02  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:16:02  |  08:16:09  |            |            |  09:07:00  |            |  0  |  0  |  1  |  3
08:16:09  |  08:16:26  |  08:16:29  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:16:26  |  08:16:43  |  08:16:29  |  08:16:56  |  09:07:00  |            |  1  |  1  |  1  |  3
08:16:29  |  08:16:4

08:29:57  |  08:30:02  |            |            |  09:07:00  |            |  0  |  0  |  1  |  3
08:30:02  |  08:30:19  |  08:30:21  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:30:19  |  08:30:36  |  08:30:21  |  08:30:49  |  09:07:00  |            |  1  |  1  |  1  |  3
08:30:21  |  08:30:36  |  08:30:41  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:30:36  |  08:30:53  |  08:30:41  |  08:31:06  |  09:07:00  |            |  1  |  1  |  1  |  3
08:30:41  |  08:30:53  |  08:30:55  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:30:53  |  08:31:10  |  08:30:55  |  08:31:23  |  09:07:00  |            |  1  |  1  |  1  |  3
08:30:55  |  08:31:10  |  08:31:06  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:31:06  |  08:31:10  |            |            |  09:07:00  |            |  0  |  0  |  1  |  3
08:31:10  |  08:31:27  |  08:31:26  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:31:26  |  08:31:2

08:43:00  |  08:43:04  |            |            |  09:07:00  |            |  0  |  0  |  1  |  3
08:43:04  |  08:43:21  |  08:43:14  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:43:14  |  08:43:21  |            |            |  09:07:00  |            |  0  |  0  |  1  |  3
08:43:21  |  08:43:38  |  08:43:38  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:43:38  |  08:43:55  |  08:43:38  |  08:44:08  |  09:07:00  |            |  1  |  1  |  1  |  3
08:43:38  |  08:43:55  |  08:43:56  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:43:55  |  08:44:12  |  08:43:56  |  08:44:25  |  09:07:00  |            |  1  |  1  |  1  |  3
08:43:56  |  08:44:12  |  08:44:11  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:44:11  |  08:44:12  |            |            |  09:07:00  |            |  0  |  0  |  1  |  3
08:44:12  |  08:44:29  |  08:44:31  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:44:29  |  08:44:4

08:56:06  |  08:56:23  |  08:56:08  |  08:56:36  |  09:07:00  |            |  1  |  1  |  1  |  3
08:56:08  |  08:56:23  |  08:56:26  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:56:23  |  08:56:40  |  08:56:26  |  08:56:53  |  09:07:00  |            |  1  |  1  |  1  |  3
08:56:26  |  08:56:40  |  08:56:40  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:56:40  |  08:56:57  |  08:56:40  |  08:57:10  |  09:07:00  |            |  1  |  1  |  1  |  3
08:56:40  |  08:56:57  |  08:57:00  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:56:57  |  08:57:14  |  08:57:00  |  08:57:27  |  09:07:00  |            |  1  |  1  |  1  |  3
08:57:00  |  08:57:14  |  08:57:13  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:57:13  |  08:57:14  |            |            |  09:07:00  |            |  0  |  0  |  1  |  3
08:57:14  |  08:57:31  |  08:57:25  |            |  09:07:00  |            |  0  |  1  |  1  |  3
08:57:25  |  08:57:3

...

10:39:48  |  10:40:05  |  10:40:06  |            |  11:09:00  |            |  0  |  1  |  1  |  7
10:40:05  |  10:40:22  |  10:40:06  |  10:40:35  |  11:09:00  |            |  1  |  1  |  1  |  7
10:40:06  |  10:40:22  |  10:40:18  |            |  11:09:00  |            |  0  |  1  |  1  |  7
10:40:18  |  10:40:22  |            |            |  11:09:00  |            |  0  |  0  |  1  |  7
10:40:22  |  10:40:39  |  10:40:34  |            |  11:09:00  |            |  0  |  1  |  1  |  7
10:40:34  |  10:40:39  |            |            |  11:09:00  |            |  0  |  0  |  1  |  7
10:40:39  |  10:40:56  |  10:40:50  |            |  11:09:00  |            |  0  |  1  |  1  |  7
10:40:50  |  10:40:56  |            |            |  11:09:00  |            |  0  |  0  |  1  |  7
10:40:56  |  10:41:13  |  10:41:13  |            |  11:09:00  |            |  0  |  1  |  1  |  7
10:41:13  |  10:41:30  |  10:41:13  |  10:41:43  |  11:09:00  |            |  1  |  1  |  1  |  7
10:41:13  |  10:41:3

10:54:45  |  10:54:49  |            |            |  11:09:00  |            |  0  |  0  |  1  |  7
10:54:49  |  10:55:06  |  10:55:01  |            |  11:09:00  |            |  0  |  1  |  1  |  7
10:55:01  |  10:55:06  |            |            |  11:09:00  |            |  0  |  0  |  1  |  7
10:55:06  |  10:55:23  |  10:55:17  |            |  11:09:00  |            |  0  |  1  |  1  |  7
10:55:17  |  10:55:23  |            |            |  11:09:00  |            |  0  |  0  |  1  |  7
10:55:23  |  10:55:40  |  10:55:42  |            |  11:09:00  |            |  0  |  1  |  1  |  7
10:55:40  |  10:55:57  |  10:55:42  |  10:56:10  |  11:09:00  |            |  1  |  1  |  1  |  7
10:55:42  |  10:55:57  |  10:55:56  |            |  11:09:00  |            |  0  |  1  |  1  |  7
10:55:56  |  10:55:57  |            |            |  11:09:00  |            |  0  |  0  |  1  |  7
10:55:57  |  10:56:14  |  10:56:16  |            |  11:09:00  |            |  0  |  1  |  1  |  7
10:56:14  |  10:56:3

11:09:50  |  11:10:07  |  11:10:11  |  11:09:46  |            |  11:10:00  |  3  |  1  |  0  |  7
11:10:00  |  11:10:07  |  11:10:11  |  11:09:46  |  12:10:00  |            |  3  |  1  |  1  |  7
11:09:46  |  11:10:07  |  11:10:11  |  11:10:03  |  12:10:00  |            |  2  |  1  |  1  |  8
11:10:03  |  11:10:07  |  11:10:11  |  11:10:20  |  12:10:00  |            |  1  |  1  |  1  |  9
11:10:07  |  11:10:24  |  11:10:11  |  11:10:20  |  12:10:00  |            |  2  |  1  |  1  |  9
11:10:11  |  11:10:24  |  11:10:27  |  11:10:37  |  12:10:00  |            |  1  |  1  |  1  |  9
11:10:24  |  11:10:41  |  11:10:27  |  11:10:37  |  12:10:00  |            |  2  |  1  |  1  |  9
11:10:27  |  11:10:41  |  11:10:41  |  11:10:54  |  12:10:00  |            |  1  |  1  |  1  |  9
11:10:41  |  11:10:58  |  11:10:41  |  11:10:54  |  12:10:00  |            |  2  |  1  |  1  |  9
11:10:41  |  11:10:58  |  11:11:01  |  11:11:11  |  12:10:00  |            |  1  |  1  |  1  |  9
11:10:58  |  11:11:1

11:31:39  |  11:31:56  |  11:31:58  |            |  12:10:00  |            |  0  |  1  |  1  |  9
11:31:56  |  11:32:13  |  11:31:58  |  11:32:26  |  12:10:00  |            |  1  |  1  |  1  |  9
11:31:58  |  11:32:13  |  11:32:14  |            |  12:10:00  |            |  0  |  1  |  1  |  9
11:32:13  |  11:32:30  |  11:32:14  |  11:32:43  |  12:10:00  |            |  1  |  1  |  1  |  9
11:32:14  |  11:32:30  |  11:32:31  |            |  12:10:00  |            |  0  |  1  |  1  |  9
11:32:30  |  11:32:47  |  11:32:31  |  11:33:00  |  12:10:00  |            |  1  |  1  |  1  |  9
11:32:31  |  11:32:47  |  11:32:43  |            |  12:10:00  |            |  0  |  1  |  1  |  9
11:32:43  |  11:32:47  |            |            |  12:10:00  |            |  0  |  0  |  1  |  9
11:32:47  |  11:33:04  |  11:32:58  |            |  12:10:00  |            |  0  |  1  |  1  |  9
11:32:58  |  11:33:04  |            |            |  12:10:00  |            |  0  |  0  |  1  |  9
11:33:04  |  11:33:2

11:54:02  |  11:54:19  |  11:54:17  |            |  12:10:00  |            |  0  |  1  |  1  |  9
11:54:17  |  11:54:19  |            |            |  12:10:00  |            |  0  |  0  |  1  |  9
11:54:19  |  11:54:36  |  11:54:39  |            |  12:10:00  |            |  0  |  1  |  1  |  9
11:54:36  |  11:54:53  |  11:54:39  |  11:55:06  |  12:10:00  |            |  1  |  1  |  1  |  9
11:54:39  |  11:54:53  |  11:54:49  |            |  12:10:00  |            |  0  |  1  |  1  |  9
11:54:49  |  11:54:53  |            |            |  12:10:00  |            |  0  |  0  |  1  |  9
11:54:53  |  11:55:10  |  11:55:06  |            |  12:10:00  |            |  0  |  1  |  1  |  9
11:55:06  |  11:55:10  |            |            |  12:10:00  |            |  0  |  0  |  1  |  9
11:55:10  |  11:55:27  |  11:55:28  |            |  12:10:00  |            |  0  |  1  |  1  |  9
11:55:27  |  11:55:44  |  11:55:28  |  11:55:57  |  12:10:00  |            |  1  |  1  |  1  |  9
11:55:28  |  11:55:4